In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage.measure import regionprops
from scipy.stats import multivariate_normal
from skimage.io import imread
import matplotlib.pyplot as plt

In [ ]:
def OAM_220820_Gaussian_nuclear_fit(IG, peak_cutoff, x_size, y_size, ccell):
    p_nuc = np.zeros(IG.shape)
    # -----gaussian fit--------------------
    I_gf = gaussian_filter(IG, sigma=2)
    nuc_tmp = I_gf * ccell
    
    Amp = np.max(nuc_tmp)
    props = regionprops(nuc_tmp > Amp * peak_cutoff, intensity_image=nuc_tmp)
    if len(props) > 0:
        centroid = props[0].centroid
        xcenter = int(centroid[1])
        ycenter = int(centroid[0])
        wind = 5
        xcoord = np.arange(max(1, xcenter - wind), min(y_size, xcenter + wind + 1))
        ycoord = np.arange(max(1, ycenter - wind), min(x_size, ycenter + wind + 1))
        X, Y = np.meshgrid(xcoord, ycoord)
        
        ok_size = wind * 2 + 1
        if X.shape == (ok_size, ok_size) and Y.shape == (ok_size, ok_size):
            In_part = nuc_tmp[Y, X]
            edge_med = np.median(np.concatenate((In_part[0, :], In_part[-1, :], In_part[:, 0], In_part[:, -1])))
            x0 = xcenter
            y0 = ycenter
            mu = np.array([x0, y0])
            best_fit_score = 1e9
            best_fit_no = np.array([0, 0])
            for jj in range(1, 26):
                for jj2 in range(1, 26):
                    limit_h = int(round(np.sqrt(jj * jj2)))
                    for ii in range(-(limit_h - 1), limit_h, 2):
                        Sigma = np.array([[jj, -ii], [-ii, jj2]])
                        F = multivariate_normal.pdf(np.column_stack((X.flatten(), Y.flatten())), mean=mu, cov=Sigma)
                        F = F.reshape(X.shape)
                        Fmax = np.max(F)
                        Z = (((Amp - edge_med) / Fmax) * F + edge_med)
                        Z = Z * (In_part > 0)
                        tmp_score = np.sum(np.abs(Z - In_part))
                        if tmp_score < best_fit_score:
                            best_fit_no = Sigma
                            best_fit_score = tmp_score
            
            # ----------------- get the best solution------------------------
            F = multivariate_normal.pdf(np.column_stack((X.flatten(), Y.flatten())), mean=mu, cov=best_fit_no)
            F = F.reshape(X.shape)
            Fmax = np.max(F)
            Z = (((Amp - edge_med) / Fmax) * F + edge_med)
            Z = Z * (In_part > 0)
            # ---------------------------------------------------------------
            p_nuc[Y.flatten(), X.flatten()] += (Z > (Amp * peak_cutoff)).astype(int)
            # ---------------------------------------------------------------------
            p_nuc = p_nuc.astype(float)
    
    return p_nuc
